



### 패스트 텍스트

In [34]:
import fasttext

In [ ]:
model.save_model('fasttext.bin')
model = fasttext.load_model('fasttext.bin')
model[word_to_jamo('남동생')]
model.get_nearest_neighbors(word_to_jamo('남동생'))

In [36]:
from konlpy.tag import Okt
from tqdm import tqdm

okt = Okt()
tokenized_data = []

with open('ratings_total.txt', 'r', encoding='utf8') as f:
    lines = f.readlines()

for line in tqdm(lines, desc="Tokenizing"):
    line = line.strip()
    if line:  # 빈 줄 제외
        tokens = okt.morphs(line)
        tokenized_data.append(tokens)


Tokenizing: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200000/200000 [05:59<00:00, 556.22it/s]


In [38]:
with open('tokenized_data.txt', 'w', encoding='utf8') as out:
    for line in tokenized_data:
        out.write(' '.join(line) + '\n')

model_cbow = fasttext.train_unsupervised('tokenized_data.txt', model='cbow')
model_cbow.save_model('fasttext_cbow.bin')

model_skipgram = fasttext.train_unsupervised('tokenized_data.txt', model='skipgram')
model_skipgram.save_model('fasttext_skipgram.bin')

words = ['남동생', '남동쉥']
results = {
    '기준단어': [],
    'CBOW_유사단어': [],
    'CBOW_유사도': [],
    'Skipgram_유사단어': [],
    'Skipgram_유사도': []
}

for target in words:
    jamo_word = word_to_jamo(target)

    cbow_neighbors = model_cbow.get_nearest_neighbors(jamo_word, k=10)
    skipgram_neighbors = model_skipgram.get_nearest_neighbors(jamo_word, k=10)

    for i in range(5):
        results['기준단어'].append(target)
        results['CBOW_유사단어'].append(cbow_neighbors[i][1])
        results['CBOW_유사도'].append(round(cbow_neighbors[i][0], 4))
        results['Skipgram_유사단어'].append(skipgram_neighbors[i][1])
        results['Skipgram_유사도'].append(round(skipgram_neighbors[i][0], 4))

df_result = pd.DataFrame(results)

,기준단어,CBOW_유사단어,CBOW_유사도,Skipgram_유사단어,Skipgram_유사도
0,남동생,맞는거죠,0.7114,베게를,0.7506
1,남동생,의심스러울,0.6715,높이는,0.6972
2,남동생,맞는지요,0.6690,낮은,0.6781
3,남동생,맞는건,0.6628,푹신한,0.6586
4,남동생,대의,0.6588,좀있는,0.6374
5,남동쉥,자아,0.7830,밝음,0.7784
6,남동쉥,고다,0.7755,와인잔,0.7783
7,남동쉥,스마일,0.7594,섬,0.7769
8,남동쉥,깰,0.7497,앞서,0.7677
9,남동쉥,그루트,0.7464,팸,0.7604
